In [3]:
from numerapi import NumerAPI
napi = NumerAPI()

# napi.download_dataset("v4.1/train.parquet", "./data/train.parquet")
# napi.download_dataset("v4.1/validation.parquet", "./data/validation.parquet")
# napi.download_dataset("v4.1/live.parquet", "../data/live.parquet")
# napi.download_dataset("v4.1/live_example_preds.parquet", "./data/live_example_preds.parquet")
# napi.download_dataset("v4.1/validation_example_preds.parquet", "./data/validation_example_preds.parquet")
# napi.download_dataset("v4.1/features.json", "./data/features.json")
# napi.download_dataset("v4.1/meta_model.parquet", "./data/meta_model.parquet")

In [4]:
### Imports
import gc, os
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import json

import joblib
from joblib import Parallel, delayed

from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale, normalize

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import math

from numerapi import NumerAPI

tqdm.pandas()

from pathlib import Path

# Paths to the downloaded datasets, model, and hyperparameters
data_dir = Path('data/')
model_dir = Path('model/')
best_hyperparams_dir = Path('best_hyperparams/')
study_dir = Path('study/')

# Create directories if they do not exist
data_dir.mkdir(parents=True, exist_ok=True)
model_dir.mkdir(parents=True, exist_ok=True)
best_hyperparams_dir.mkdir(parents=True, exist_ok=True)
study_dir.mkdir(parents=True, exist_ok=True)

In [5]:
with open(data_dir / "features.json") as f:
    features_meta = json.load(f)

features_to_load = features_meta["feature_sets"]["small"] # medium or all features

live = pd.read_parquet(data_dir / "live.parquet")

cols_to_ignore = [
    c for c in live.columns if (c not in features_to_load and "feature_" in c)
]
cols_to_load = [c for c in live.columns if c not in cols_to_ignore]

train = pd.read_parquet(data_dir / "train.parquet", columns=cols_to_load)
validation = pd.read_parquet(data_dir / "validation.parquet", columns=cols_to_load)
live = pd.read_parquet(data_dir / "live.parquet", columns=cols_to_load)


live_example_preds = pd.read_parquet(data_dir / "live_example_preds.parquet")
validation_example_preds = pd.read_parquet(data_dir / "validation_example_preds.parquet")
meta_model = pd.read_parquet(data_dir / "meta_model.parquet")

assert validation.shape[0] == validation_example_preds.shape[0]


# # Convert era to integer
# train["era_int"] = train["era"].astype(int)
# validation["era_int"] = validation["era"].astype(int)
# gc.collect()

# # # assert live.shape[0] == live_example_preds.shape[0]
# # assert validation.shape[0] == meta_model.shape[0]


feature_names = [f for f in train.columns if "feature_" in f]
feature_names = [f for f in feature_names if f in features_to_load]
target_names = [t for t in train.columns if "target_" in t] #[:5]

# TARGET_NAME = target_names[0]
# PREDICTION_NAME = "prediction"

In [6]:
TARGET_NAME = target_names[0]
PREDICTION_NAME = "prediction"

# # all eras of TARGET_NAME must be present in diagnostics
# validation = validation.dropna(subset=[TARGET_NAME], axis=0).copy()
# # validation[TARGET_NAME] = validation[TARGET_NAME].fillna(0.5)

# Get the indices where TARGET_NAME is not NaN
valid_indices = validation[~validation[TARGET_NAME].isna()].index

# Use these indices to subset validation
validation = validation.loc[valid_indices].copy()

# Use the same indices to subset validation_example_preds
validation_example_preds = validation_example_preds.loc[valid_indices]

assert validation.shape[0] == validation_example_preds.shape[0]

gc.collect()

# gc.collect()

train["era_int"] = train["era"].astype(int)
validation["era_int"] = validation["era"].astype(int)
gc.collect()

train[feature_names] = train[feature_names].fillna(-2)
validation[feature_names] = validation[feature_names].fillna(-2)
gc.collect()

# use a better method to handle NaN in targets
train[target_names] = train[target_names].fillna(0.5)
validation[target_names] = validation[target_names].fillna(0.5)
gc.collect()

# device = "cpu"

0

In [7]:
PADDING_VALUE = -1
MAX_LEN = 6000
FEATURE_DIM = len(feature_names)
HIDDEN_DIM = 128
OUTPUT_DIM = len(target_names)
NUM_HEADS = 2
NUM_LAYERS = 2

device = "cuda" if torch.cuda.is_available() else "cpu"

In [26]:
device

'cuda'

In [8]:
def pad_sequence(inputs, padding_value=-1, max_len=None):
    if max_len is None:
        max_len = max([input.shape[0] for input in inputs])
    padded_inputs = []
    masks = []
    for input in inputs:
        pad_len = max_len - input.shape[0]
        padded_input = F.pad(input, (0, 0, 0, pad_len), value=padding_value)
        mask = torch.ones((input.shape[0], 1), dtype=torch.float)
        masks.append(
            torch.cat((mask, torch.zeros((pad_len, 1), dtype=torch.float)), dim=0)
        )
        padded_inputs.append(padded_input)
    return torch.stack(padded_inputs), torch.stack(masks)


def convert_to_torch(era, data):

    inputs = torch.from_numpy(
                data[feature_names].values.astype(np.int8))
    labels = torch.from_numpy(
                data[target_names].values.astype(np.float32))

    padded_inputs, masks_inputs = pad_sequence(
            [inputs], padding_value=PADDING_VALUE, max_len=MAX_LEN)
    padded_labels, masks_labels = pad_sequence(
            [labels], padding_value=PADDING_VALUE, max_len=MAX_LEN)

    return {
        era: (
            padded_inputs, # features
            padded_labels,
            masks_inputs
        )
    }

def get_era2data(df):
    res = Parallel(n_jobs=-1, prefer="threads")(
        delayed(convert_to_torch)(era, data)
        for era, data in tqdm(df.groupby("era_int")))
    era2data = {}
    for r in tqdm(res):
        era2data.update(r)
    return era2data

## dataloader
era2data_train = get_era2data(train)
era2data_validation = get_era2data(validation)

  0%|          | 0/574 [00:00<?, ?it/s]

100%|██████████| 492/492 [00:00<00:00, 1930399.97it/s]


In [9]:
from model import Transformer

def test_model():

    inputs = [
        torch.randint(0, 4, (5, FEATURE_DIM)).float(),
        torch.randint(0, 4, (3, FEATURE_DIM)).float(),
    ]
    labels = [
        torch.randint(0, 2, (5, OUTPUT_DIM)).float(),
        torch.randint(0, 2, (3, OUTPUT_DIM)).float(),
    ]

    padded_inputs, masks_inputs = pad_sequence(inputs, \
                                               padding_value=0, max_len=MAX_LEN)
    padded_labels, masks_labels = pad_sequence(labels, \
                                               padding_value=0, max_len=MAX_LEN)

    transformer = Transformer(
        input_dim=FEATURE_DIM,
        d_model=HIDDEN_DIM,
        output_dim=OUTPUT_DIM,
        num_heads=NUM_HEADS,
        num_layers=NUM_LAYERS,
        max_len=MAX_LEN,
    )

    with torch.no_grad():
        outputs = transformer(padded_inputs, masks_inputs)

    assert torch.isnan(outputs).sum() == 0
    assert outputs.shape[:2] == padded_inputs.shape[:2]
    assert outputs.shape[-1] == len(target_names)

    print("Input Shape:", padded_inputs.shape)
    print("Output Shape:", outputs.shape)

    del transformer
    del inputs, labels
    del padded_inputs, masks_inputs, padded_labels, masks_labels
    del outputs

    gc.collect()

test_model()

Input Shape: torch.Size([2, 6000, 32])
Output Shape: torch.Size([2, 6000, 36])


In [10]:
# pearsonr in torch differentiable
def pearsonr(x, y):
    mx = x.mean()
    my = y.mean()
    xm, ym = x - mx, y - my
    r_num = torch.sum(xm * ym)
    r_den = torch.sqrt(torch.sum(xm ** 2) * torch.sum(ym ** 2))
    r = r_num / r_den
    return r


In [11]:
def calculate_loss(outputs, criterion, padded_labels, masks_inputs, \
                padded_inputs=None, target_weight_softmax=None):

    # MSE on all targets; additionally, on primary target
    if target_weight_softmax is not None:
        _mse = criterion(
            outputs * masks_inputs * target_weight_softmax,
            padded_labels * masks_inputs * target_weight_softmax
        ) * 0.1

    else:
        _mse = criterion(outputs * masks_inputs, padded_labels * masks_inputs) * 0.1

    _mse += criterion(outputs[:, 0] * masks_inputs, padded_labels[:, 0] * masks_inputs)

    # Corr with only primary target; adjust as needed
    corr = pearsonr(
        outputs[0][:, 0][masks_inputs.view(-1).nonzero()].view(-1, 1),
        padded_labels[0][:, 0][masks_inputs.view(-1).nonzero()].view(-1, 1),
    )

    loss = _mse - corr #+ some_complex_constraints
    return loss, _mse, corr

# Training loop
def train_on_batch(transformer, criterion, optimizer, batch):

    padded_inputs = batch[0].to(device=device)
    padded_labels = batch[1].to(device=device)
    masks_inputs = batch[2].to(device=device)

    optimizer.zero_grad()

    outputs = transformer(padded_inputs / 4.0, masks_inputs)
    # print(outputs)

    target_weight_softmax = None
    #random_weights = torch.rand(padded_labels.shape[-1], device=device)
    #target_weight_softmax = F.softmax(random_weights)

    loss, _mse, _corr = calculate_loss(outputs, criterion, padded_labels, masks_inputs, \
                                       target_weight_softmax=target_weight_softmax)
    loss.backward()
    optimizer.step()
    return loss.item(), _mse.item(), _corr.item()


def evaluate_on_batch(transformer, criterion, batch):

    padded_inputs = batch[0].to(device=device)
    padded_labels = batch[1].to(device=device)
    masks_inputs = batch[2].to(device=device)

    transformer.eval()
    with torch.no_grad():
        outputs = transformer(padded_inputs / 4.0, masks_inputs)
        # print(outputs)
        loss, _mse, _corr = calculate_loss(outputs, criterion, padded_labels, masks_inputs)
        
        # Convert outputs to numpy
        preds = outputs[0][masks_inputs.view(-1).nonzero()].squeeze(1).cpu().numpy()
        # print(preds)

    return loss.item(), _mse.item(), _corr.item(), preds


def metrics_on_batch(era_scores):
    era_scores = pd.Series(era_scores)
    
    # Calculate metrics
    mean_correlation = np.mean(era_scores)
    std_deviation = np.std(era_scores)
    sharpe_ratio = mean_correlation / std_deviation
    max_dd = (era_scores.cummax() - era_scores).max() # from calculate_metrics

    # Smart Sharpe: Modified Sharpe ratio that also considers the instability of scores over time,
    # penalizing models with high score instability even if their mean score is high
    smart_sharpe = mean_correlation / (std_deviation + np.std(era_scores.diff()))
    
    # Autocorrelation: Measure of the correlation of the series with a lagged version of itself
    autocorrelation = era_scores.autocorr()

    metrics = pd.Series({
        'mean_correlation': mean_correlation,
        'std_deviation': std_deviation,
        'sharpe_ratio': sharpe_ratio,
        'smart_sharpe': smart_sharpe,
        'autocorrelation': autocorrelation,
        'max_dd': max_dd, # added from calculate_metrics
        'min_correlation': era_scores.min(), # added from calculate_metrics
        'max_correlation': era_scores.max(), # added from calculate_metrics
    })

    # Cleanup
    _ = gc.collect()
    
    return metrics



In [12]:
from tqdm.auto import tqdm

def train_model(transformer, criterion, optimizer, scheduler, \
                num_epochs, patience, train_loader, val_loader, is_lr_scheduler=True):
    best_loss = float('inf')
    best_corr = None
    best_model = None
    best_outputs = None
    no_improve_epoch = 0

    epoch_progress = tqdm(range(num_epochs), desc="Epochs", position=0, leave=False)

    for epoch in epoch_progress:
        total_loss = []
        total_corr = []

        # Training
        for era_num in tqdm(train_loader, desc="Training", leave=False, position=1):
            batch = train_loader[era_num]
            loss, _mse, _corr = train_on_batch(transformer, criterion, optimizer, batch)
            total_loss.append(loss)
            total_corr.append(_corr)

        # Adjust learning rate if is_lr_scheduler is True
        if is_lr_scheduler:
            scheduler.step()

        # Validation
        transformer.eval()
        val_total_loss = []
        val_total_corr = []
        val_total_outputs = {}
        with torch.no_grad():
            for era_num in tqdm(val_loader, desc="Validation", leave=False, position=2):
                batch = val_loader[era_num]
                loss, _mse, _corr, outputs = evaluate_on_batch(transformer, criterion, batch)
                # print(outputs)
                val_total_loss.append(loss)
                val_total_corr.append(_corr)
                val_total_outputs[era_num] = outputs

        # Early stopping check
        val_loss = np.mean(val_total_loss)
        if val_loss < best_loss:
            best_loss = val_loss
            best_corr = val_total_corr.copy()
            best_model = transformer.state_dict().copy()
            best_outputs = val_total_outputs.copy()
            no_improve_epoch = 0
        else:
            no_improve_epoch += 1
            if no_improve_epoch >= patience:
                epoch_progress.set_description(f'Early stopping at epoch {epoch+1}')
                epoch_progress.refresh()
                break

        torch.cuda.empty_cache()
        _ = gc.collect()

    # Save the best model state
    torch.save(best_model, data_dir / "transformer_best.pth")

    return transformer, best_corr, best_outputs # best_outputs for future use


In [13]:
# # Define the model, loss function, and optimizer

# gc.collect()
# transformer = Transformer(
#     input_dim=FEATURE_DIM,
#     d_model=HIDDEN_DIM,
#     output_dim=OUTPUT_DIM,
#     num_heads=NUM_HEADS,
#     num_layers=NUM_LAYERS,
# )

# # load model from checkpoint
# if (data_dir / "transformer.pth").is_file():
#     transformer.load_state_dict(torch.load(data_dir / "transformer.pth"))

# transformer.to(device=device)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(transformer.parameters(), lr=1e-4)

# # Number of training iterations
# # Train for longer with low LR

# num_epochs = 1
# patience = 5

# from torch.optim.lr_scheduler import StepLR
# scheduler = StepLR(optimizer, step_size=100, gamma=0.1)

# transformer, best_corr, preds = train_model(transformer, criterion, optimizer, scheduler, \
#                                     num_epochs, patience, era2data_train, \
#                                     era2data_validation, is_lr_scheduler=True)

In [14]:
def add_predictions_and_merge(validation, val_total_outputs, validation_example_preds, meta_model):
    
    validation["example_preds"] = validation_example_preds
    # Initialize an empty list to store predictions
    all_predictions = []

    # Iterate over sorted era numbers
    for era_num in sorted(val_total_outputs.keys()):
        # Get the predictions for the current era
        predictions = val_total_outputs[era_num]
        
        # We only need the first column of each prediction
        predictions = predictions[:, 0]
        
        # Add the predictions to the list
        all_predictions.extend(predictions)

    # Now, all_predictions is a single list containing all predictions in order
    # Convert it to a numpy array
    all_predictions = np.array(all_predictions)

    # Add the predictions as a new column to the validation DataFrame
    validation[PREDICTION_NAME] = all_predictions
    

    # Reset the index and only keep the required columns
    diagnosis = validation[["era_int", PREDICTION_NAME, \
                            TARGET_NAME, "example_preds"]].reset_index()

    # Merge diagnosis with meta model
    combined = pd.merge(
        diagnosis,
        meta_model,
        on=["id"],
        how="right"
    ).dropna(axis=0)

    return combined


def unif(df):
    """from example scripts"""
    x = (df.rank(method="first") - 0.5) / len(df)
    return pd.Series(x, index=df.index)


def calculate_correlations(combined, PREDICTION_NAME):

    pred2meta_model_corr = (
        combined[["era_int", "numerai_meta_model", PREDICTION_NAME]]
        .groupby("era_int")
        .progress_apply(
            lambda d: unif(d[PREDICTION_NAME]).corr(d["numerai_meta_model"])
        )
        .sort_index()
        .rename("pred2meta_model_corr")
    )

    example_preds_corr = (
        combined[["era_int", "example_preds", PREDICTION_NAME]]
        .groupby("era_int")
        .progress_apply(
            lambda d: unif(d[PREDICTION_NAME]).corr(d["example_preds"])
        )
        .sort_index()
        .rename("pred2example_preds_corr")
    )

    example2meta_model_corr = (
        combined[["era_int", "example_preds", "numerai_meta_model"]]
        .groupby("era_int")
        .progress_apply(
            lambda d: unif(d["numerai_meta_model"]).corr(d["example_preds"])
        )
        .sort_index()
        .rename("example2meta_model_corr")
    )

    return pred2meta_model_corr, example_preds_corr, example2meta_model_corr


In [15]:
# import optuna
# from torch.optim.lr_scheduler import StepLR

# NUM_EPOCHS = 1
# PATIENCE = 5

# def objective(trial, validation=validation, validation_example_preds=validation_example_preds, \
#               meta_model=meta_model):
#     # Define the model, loss function, and optimizer
#     print(f"\n--- Starting Trial: {trial.number + 1} ---")

#     _ = gc.collect()

#     # suggest hyperparameters
#     num_heads = trial.suggest_int("num_heads", 1, 5)
#     hidden_dim = trial.suggest_int("hidden_dim", 64, 256, step=2)
#     num_layers = trial.suggest_int("num_layers", 1, 5)
#     lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    
#     transformer = Transformer(
#         input_dim=FEATURE_DIM,
#         d_model=hidden_dim,
#         output_dim=OUTPUT_DIM,
#         num_heads=num_heads,
#         num_layers=num_layers,
#     )

#     # # load model from checkpoint
#     # if (model_dir / "transformer.pth").is_file():
#     #     transformer.load_state_dict(torch.load(model_dir / "transformer.pth"))

#     transformer.to(device=device)
#     criterion = nn.MSELoss()
#     optimizer = optim.Adam(transformer.parameters(), lr=lr)
#     scheduler = StepLR(optimizer, step_size=100, gamma=0.1)

#     # Number of training iterations
#     num_epochs = NUM_EPOCHS 
#     patience = PATIENCE
#     transformer, best_corr, outputs = train_model(transformer, criterion, optimizer, scheduler, \
#                                         num_epochs, patience, era2data_train, \
#                                         era2data_validation, is_lr_scheduler=True)
    
#     # # print(f'Validation shape: {validation.shape}')
#     # print(outputs)
#     metrics = metrics_on_batch(best_corr)

#     trial.set_user_attr("transformer", transformer)


#     # diagnostic = add_predictions_and_merge(validation, outputs, validation_example_preds, meta_model)
#     # pred2meta_model_corr, example_preds_corr, example2meta_model_corr = \
#     #             calculate_correlations(diagnostic, PREDICTION_NAME)
#     # # print(f'diagnostic shape after: {diagnostic.shape} and col names: {diagnostic.columns}')

#     ### Save model and parameters if it's the best so far
#     # if trial.value is None or trial.value < study.best_value:
#     #     torch.save(transformer, model_dir / f"best_model_trial_{trial.number}.pth")
#     #     with open(best_hyperparams_dir / f"best_params_trial_{trial.number}.json", 'w') as f:
#     #         json.dump(trial.params, f)
    
#     return -metrics['sharpe_ratio']  # return negative correlation because optuna minimizes the objective function

# # Callback to save the best model and its hyperparameters after each trial
# def callback(study, trial):
#     print(f"\n--- Trial {trial.number + 1} finished ---")
#     print(f"Value: {trial.value} and parameters: {trial.params}")
#     print(f"Best is trial {study.best_trial.number} with value: {study.best_trial.value}\n")
    
#     # If the trial is the best so far, save the model and hyperparameters
#     if study.best_trial.number == trial.number:
#         best_model = trial.user_attrs["transformer"]
#         torch.save(best_model, model_dir / "best_model.pth")
#         with open(best_hyperparams_dir / "best_params.json", 'w') as f:
#             json.dump(trial.params, f)

# study = optuna.create_study(study_name='Maximizing the Sharpe', direction='minimize', \
#                             storage=f'sqlite:///{study_dir}/study.db', load_if_exists=True)
# study.optimize(objective, n_trials=25, callbacks=[callback])
# gc.collect()  # manual garbage collection

In [16]:
import optuna
from torch.optim.lr_scheduler import StepLR

NUM_EPOCHS = 15
PATIENCE = 5


def objective(trial, validation=validation, validation_example_preds=validation_example_preds, \
              meta_model=meta_model):
    # Define the model, loss function, and optimizer
    print(f"\n--- Starting Trial: {trial.number + 1} ---")

    _ = gc.collect()

    # suggest hyperparameters
    num_heads = trial.suggest_int("num_heads", 1, 5)
    hidden_dim = trial.suggest_int("hidden_dim", 64, 256, step=2)
    num_layers = trial.suggest_int("num_layers", 1, 5)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    
    transformer = Transformer(
        input_dim=FEATURE_DIM,
        d_model=hidden_dim,
        output_dim=OUTPUT_DIM,
        num_heads=num_heads,
        num_layers=num_layers,
    )

    # # load model from checkpoint
    # if (model_dir / "transformer.pth").is_file():
    #     transformer.load_state_dict(torch.load(model_dir / "transformer.pth"))

    transformer.to(device=device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(transformer.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=100, gamma=0.1)

    # Number of training iterations
    num_epochs = NUM_EPOCHS 
    patience = PATIENCE
    transformer, best_corr, outputs = train_model(transformer, criterion, optimizer, scheduler, \
                                        num_epochs, patience, era2data_train, \
                                        era2data_validation, is_lr_scheduler=True)
    
    metrics = metrics_on_batch(best_corr)

    # diagnostic = add_predictions_and_merge(validation, outputs, validation_example_preds, meta_model)
    # pred2meta_model_corr, example_preds_corr, example2meta_model_corr = \
    #             calculate_correlations(diagnostic, PREDICTION_NAME)
    
    # # print(f'diagnostic shape after: {diagnostic.shape} and col names: {diagnostic.columns}')

    # Save the transformer model after each trial

    torch.save(transformer, model_dir / f"model_trial_{trial.number}.pth")

    return -metrics['sharpe_ratio']  # return negative correlation because optuna minimizes the objective function


# Callback to save the best model and its hyperparameters after each trial
def callback(study, trial):
    print(f"\n--- Trial {trial.number + 1} finished ---")
    print(f"Value: {trial.value} and parameters: {trial.params}")
    print(f"Best is trial {study.best_trial.number} with value: {study.best_trial.value}\n")
    
    # If the trial is the best so far, rename the saved model file and save the hyperparameters
    if study.best_trial.number == trial.number:
        # Rename the saved model file
        os.rename(model_dir / f"model_trial_{trial.number}.pth", model_dir / "best_model.pth")

        # Save the hyperparameters
        with open(best_hyperparams_dir / "best_params.json", 'w') as f:
            json.dump(trial.params, f)

In [17]:
#### Uncomment to train #####

# study = optuna.create_study(study_name='Maximizing the Sharpe', direction='minimize', \
#                             storage=f'sqlite:///{study_dir}/study.db', load_if_exists=True)

# study.optimize(objective, n_trials=25, callbacks=[callback])
# gc.collect()  # manual garbage collection

In [18]:
def analyze_model_performance(pred2meta_model_corr, example_preds_corr, example2meta_model_corr):
    # Use the metrics to generate a qualitative assessment of the model's performance
    print("=== Model Performance Analysis ===")

    # Analyze correlation to the meta model
    if pred2meta_model_corr.mean() > 0:
        print(f"Positively correlated with the meta model. Average correlation: {pred2meta_model_corr.mean():.4f}.")
    else:
        print(f"Negatively correlated with the meta model. Average correlation: {pred2meta_model_corr.mean():.4f}.")

    # Analyze correlation to the example predictions
    if example_preds_corr.mean() > 0:
        print(f"Positively correlated with the example predictions. Average correlation: {example_preds_corr.mean():.4f}.")
    else:
        print(f"Negatively correlated with the example predictions. Average correlation: {example_preds_corr.mean():.4f}.")

    # Analyze correlation between the meta model and the example predictions
    if example2meta_model_corr.mean() > 0:
        print(f"Meta model's predictions are positively correlated with the example predictions. Average correlation: {example2meta_model_corr.mean():.4f}.")
    else:
        print(f"Meta model's predictions are negatively correlated with the example predictions. Average correlation: {example2meta_model_corr.mean():.4f}.")

    print("=== End of Model Performance Analysis ===")


In [19]:
model = torch.load(model_dir / "best_model.pth")

In [20]:
def predict_on_era2data(era2data, transformer, criterion, device="cuda"):
    """
    Currently returns only primary target.
    outputs[0][:, 0]: target_nomi_v4_20
    """
    transformer.eval()

    val_total_loss = []
    val_total_corr = []
    val_total_outputs = {}

    with torch.no_grad():
        for era_num in tqdm(era2data, desc="Validation", leave=False, position=2):
            batch = era2data[era_num]

            # Move to specified device
            batch = (batch[0].to(device=device), batch[1].to(device=device), batch[2].to(device=device))

            # evaluate_on_batch should return loss, mse, corr, outputs
            loss, _mse, _corr, outputs = evaluate_on_batch(transformer, criterion, batch)

            # Store metrics
            val_total_loss.append(loss)
            val_total_corr.append(_corr)
            val_total_outputs[era_num] = outputs

    return val_total_loss, val_total_corr, val_total_outputs

criterion = nn.MSELoss()
_, _, outputs = predict_on_era2data(era2data_validation, model, criterion, device="cuda")

Validation:   0%|          | 0/492 [00:00<?, ?it/s]

In [21]:
diagnostic = add_predictions_and_merge(validation, outputs, validation_example_preds, meta_model)
pred2meta_model_corr, example_preds_corr, example2meta_model_corr = \
            calculate_correlations(diagnostic, PREDICTION_NAME)
    

100%|██████████| 179/179 [00:00<00:00, 1998.26it/s]


In [22]:
analyze_model_performance(pred2meta_model_corr, example_preds_corr, example2meta_model_corr)

=== Model Performance Analysis ===
Positively correlated with the meta model. Average correlation: 0.4074.
Positively correlated with the example predictions. Average correlation: 0.2702.
Meta model's predictions are positively correlated with the example predictions. Average correlation: 0.6330.
=== End of Model Performance Analysis ===


In [23]:
transformer = model

In [24]:
def prepare_live(data):

    inputs = torch.from_numpy(
                data[feature_names].values.astype(np.int8))

    padded_inputs, masks_inputs = pad_sequence(
            [inputs], padding_value=PADDING_VALUE, max_len=MAX_LEN)

    return padded_inputs, masks_inputs

In [25]:
# Wrap your model with a function that takes live features and returns live predictions
def predict(live_features: pd.DataFrame) -> pd.DataFrame:

    device = "cpu"
    transformer.to(device)
    if 0.5 in live_features[feature_names[0]]:
        live_features[feature_names] = (live_features[feature_names] * 4.0).astype(int)

    input_live, mask_live = prepare_live(live_features)
    input_live, mask_live = input_live.to(device), mask_live.to(device)
    preds_live = transformer(input_live/4.0, mask_live)[0][mask_live.view(-1).nonzero()].squeeze(1).detach().cpu().numpy()[:, 0]
    print(preds_live.shape)
    submission = pd.Series(preds_live, index=live_features.index)
    return submission.to_frame("prediction")

# Use the cloudpickle library to serialize your function
import cloudpickle
p = cloudpickle.dumps(predict)
with open("predict_transformer.pkl", "wb") as f:
    f.write(p)